In [1]:
import json
import requests
import requests_cache
import pandas as pd
import pprint
import numpy as np
import isodbtools

requests_cache.install_cache('api_map_cache')

# Download Helpful Data Files

In [2]:
# Build a Library of Adsorptives
#   NOTE: this will accelerate name <-> InChIKey mapping
URL = 'https://adsorption.nist.gov/isodb/api/gases.json'
adsorptives = requests.get(URL).json()

# Build a Library of Adsorbents
URL = 'https://adsorption.nist.gov/matdb/api/materials.json'
adsorbents = requests.get(URL).json()

# Download the (abbreviated) Isotherm Library
URL = 'https://adsorption.nist.gov/isodb/api/isotherms.json'
isotherms = requests.get(URL).json()

# Define Search Criteria

In [14]:
# Adsorptive Species
adsorptive = 'CO2'
adsorptive = requests.get('https://adsorption.nist.gov/isodb/api/gas/'+adsorptive+'.json').json()
pprint.pprint(adsorptive)
target_InChIKey = adsorptive['InChIKey']

# Temperature Range
temperature_min = 273 #K
temperature_max = 323 #K

{'InChICode': 'InChI=1S/CO2/c2-1-3',
 'InChIKey': 'CURLTUGMZLYLDI-UHFFFAOYSA-N',
 'formula': 'CO2',
 'name': 'Carbon Dioxide',
 'synonyms': ['Carbon dioxide', 'Carbon oxide', 'CO2']}


# Filter Isotherms Library against first-level criteria

In [17]:
candidate_isotherms = []

for index,isotherm in enumerate(isotherms):
    # Select single-component isotherms with desired adsorptive
    if len(isotherm['adsorbates']) == 1 and isotherm['adsorbates'][0]['InChIKey'] == target_InChIKey:
        if temperature_min <= isotherm['temperature'] <= temperature_max:
            #print(isotherm['filename'], isotherm['temperature'])
            candidate_isotherms.append(isotherm['filename'])
    #if index == 1000: break

# Filter Isotherms against second-level criteria

In [18]:
for index,filename in enumerate(candidate_isotherms):
    URL = 'https://adsorption.nist.gov/isodb/api/isotherm/'+filename+'.json&k=dontrackmeplease'
    isotherm_full = requests.get(URL).json()
    pprint.pprint(isotherm_full)
    
    
    if index == 0: break

{'DOI': '10.1002/adfm.201101479',
 'adsorbates': [{'InChIKey': 'CURLTUGMZLYLDI-UHFFFAOYSA-N',
                 'name': 'Carbon Dioxide'}],
 'adsorbent': {'hashkey': 'NIST-MATDB-655315f090663b4723680a6683f06403',
               'name': 'Zn(BDC)(DMBPY) 0.5'},
 'adsorptionUnits': 'wt%',
 'articleSource': 'UNKNOWN',
 'category': '',
 'compositionType': 'molefraction',
 'concentrationUnits': '',
 'date': '1000-01-01',
 'digitizer': 'UNKNOWN',
 'filename': '10.1002adfm.201101479.Isotherm13',
 'isotherm_data': [{'pressure': 0.0323955,
                    'species_data': [{'InChIKey': 'CURLTUGMZLYLDI-UHFFFAOYSA-N',
                                      'adsorption': 0.363636,
                                      'composition': 1}],
                    'total_adsorption': 0.363636},
                   {'pressure': 0.0500395,
                    'species_data': [{'InChIKey': 'CURLTUGMZLYLDI-UHFFFAOYSA-N',
                                      'adsorption': 0.454545,
                            